<a href="https://colab.research.google.com/github/akash8190/unification-documentation-1/blob/main/machine%20translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Convert a word written in English script to Hindi using deep learning**

In [51]:
#Make imports
import numpy as np
import re
import pickle
import os
import seaborn as sns
import string

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model


In [3]:
def preprocess(text):
  text = ''.join(ch for ch in text if ch not in string.punctuation)
  text = text.lower()
  text = re.sub(r'\d','',text)
  text = re.sub(r'\s+',' ',text)
  text = text.strip()
  return text
     

In [6]:
import pandas as pd

# Read the XML file into a DataFrame
training_data = pd.read_xml('/NEWS2018_M-EnHi_trn.xml')

# Print the DataFrame
print(training_data)

          ID                 SourceName                      TargetName
0          1                     aabhaa                             आभा
1          2                    aabheer                            आभीर
2          3                  aabhijaat                          आभिजात
3          4                      aabid                            आबिद
4          5                    aabshar                            आबशर
...      ...                        ...                             ...
12932  12933        zulm ko jala doonga             ज़ुल्म को जला दूँगा
12933  12934                  zunheboto                      जुन्हेबोटो
12934  12935  zurich financial services  ज़्यूरिक फ़ाइनेंशियल सर्विसेज़
12935  12936                      zurna                          ज़ुर्ना
12936  12937                 zwyndrecht                     ज़ॉएन्ड्रेच

[12937 rows x 3 columns]


In [7]:
training_data.head(50)

,ID,SourceName,TargetName
0,1,aabhaa,आभा
1,2,aabheer,आभीर
2,3,aabhijaat,आभिजात
3,4,aabid,आबिद
4,5,aabshar,आबशर
5,6,aachaaryanandan,आचार्यनंदन
6,7,aachaaryanandanaa,आचार्यनंदना
7,8,aachaaryasut,आचार्यसुत
8,9,aachaaryasuta,आचार्यसुता
9,10,aachaaryasutaa,आचार्यसुता


In [8]:
import pandas as pd

# Read the XML file into a DataFrame
testing_data = pd.read_xml('/NEWS2018_M-EnHi_dev.xml')

# Print the DataFrame
print(testing_data)

       ID        SourceName   TargetName
0       1  aachaaryanandana  आचार्यनंदना
1       2       aachaarysut    आचार्यसुत
2       3     aacharynandan   आचार्यनंदन
3       4   aacharynandanaa  आचार्यनंदना
4       5         aadamkhor       आदमखोर
..    ...               ...          ...
995   996      yuvraj singh  युवराज सिंह
996   997      zahoor elahi   ज़हूर इलाही
997   998            zaleel         ज़लील
998   999              zayb          ज़ैब
999  1000            zorion      ज़ोरियन

[1000 rows x 3 columns]


In [10]:
testing_data.head(50)


,ID,SourceName,TargetName
0,1,aachaaryanandana,आचार्यनंदना
1,2,aachaarysut,आचार्यसुत
2,3,aacharynandan,आचार्यनंदन
3,4,aacharynandanaa,आचार्यनंदना
4,5,aadamkhor,आदमखोर
5,6,aag aur shola,आग और शोला
6,7,aaj ki baat,आज की बात
7,8,aaj ki taza khabar,आज की ताज़ा खबर
8,9,aakanksha,आकांक्षा
9,10,aan,आन


In [11]:

pd.isnull(training_data).sum()

ID            0
SourceName    0
TargetName    0
dtype: int64

In [13]:
training_data.drop_duplicates(inplace=True)

In [15]:
training_data.shape

(12937, 3)

In [19]:
# Lowercase all characters
training_data['SourceName']=training_data['SourceName'].apply(lambda x: x.lower())
training_data['TargetName']=training_data['TargetName'].apply(lambda x: x.lower())

In [20]:
training_data['SourceName']=training_data['SourceName'].apply(lambda x: re.sub("'", '', x))
training_data['TargetName']=training_data['TargetName'].apply(lambda x: re.sub("'", '', x))
#remove quotes

In [21]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
training_data['SourceName']=training_data['SourceName'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
training_data['TargetName']=training_data['TargetName'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [25]:


training_data['TargetName'] = training_data['TargetName'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
training_data['SourceName']=training_data['SourceName'].apply(lambda x: x.strip())
training_data['TargetName']=training_data['TargetName'].apply(lambda x: x.strip())
training_data['SourceName']=training_data['SourceName'].apply(lambda x: re.sub(" +", " ", x))
training_data['TargetName']=training_data['TargetName'].apply(lambda x: re.sub(" +", " ", x))

In [26]:
# Add start and end tokens to target sequences
training_data['TargetName'] = training_data['TargetName'].apply(lambda x : 'START_ '+ x + ' _END')

In [27]:
training_data.head()

,ID,SourceName,TargetName
0,1,aabhaa,START_ आभा _END
1,2,aabheer,START_ आभीर _END
2,3,aabhijaat,START_ आभिजात _END
3,4,aabid,START_ आबिद _END
4,5,aabshar,START_ आबशर _END


In [29]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in training_data['SourceName']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in training_data['TargetName']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [30]:
len(all_eng_words)

13411

In [31]:
len(all_hindi_words)

12981

In [33]:
training_data['length_eng_sentence']=training_data['SourceName'].apply(lambda x:len(x.split(" ")))
training_data['length_hin_sentence']=training_data['TargetName'].apply(lambda x:len(x.split(" ")))

In [34]:
training_data.head()

,ID,SourceName,TargetName,length_eng_sentence,length_hin_sentence
0,1,aabhaa,START_ आभा _END,1,3
1,2,aabheer,START_ आभीर _END,1,3
2,3,aabhijaat,START_ आभिजात _END,1,3
3,4,aabid,START_ आबिद _END,1,3
4,5,aabshar,START_ आबशर _END,1,3


In [35]:
training_data[training_data['length_eng_sentence']>30].shape

(0, 5)

In [36]:
training_data=training_data[training_data['length_eng_sentence']<=20]
training_data=training_data[training_data['length_hin_sentence']<=20]

In [38]:
training_data.shape

(12937, 5)

In [39]:
print("maximum length of Hindi Sentence ",max(training_data['length_hin_sentence']))
print("maximum length of English Sentence ",max(training_data['length_eng_sentence']))

maximum length of Hindi Sentence  12
maximum length of English Sentence  10


In [41]:
max_length_src=max(training_data['length_hin_sentence'])
max_length_tar=max(training_data['length_eng_sentence'])

In [42]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(13411, 12981)

In [43]:
num_decoder_tokens += 1 #for zero padding

In [44]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [45]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [48]:

training_data.head(10)


,ID,SourceName,TargetName,length_eng_sentence,length_hin_sentence
0,1,aabhaa,START_ आभा _END,1,3
1,2,aabheer,START_ आभीर _END,1,3
2,3,aabhijaat,START_ आभिजात _END,1,3
3,4,aabid,START_ आबिद _END,1,3
4,5,aabshar,START_ आबशर _END,1,3
5,6,aachaaryanandan,START_ आचार्यनंदन _END,1,3
6,7,aachaaryanandanaa,START_ आचार्यनंदना _END,1,3
7,8,aachaaryasut,START_ आचार्यसुत _END,1,3
8,9,aachaaryasuta,START_ आचार्यसुता _END,1,3
9,10,aachaaryasutaa,START_ आचार्यसुता _END,1,3


**Split the data into train and test**

In [52]:
X, y = training_data['SourceName'], training_data['TargetName']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((10349,), (2588,))

In [53]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [54]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [55]:
latent_dim=300

In [56]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [57]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [58]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [59]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    4023300     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    3894600     ['input_2[0][0]']                
                                                                                              

In [62]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [63]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50
 1/80 [..............................] - ETA: 6s - loss: 1.5166

<ipython-input-63-b22c3ec5e69e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


80/80 [==============================] - ETA: 0s - loss: 1.3968

80/80 [==============================] - 5s 62ms/step - loss: 1.3968
Epoch 2/50
80/80 [==============================] - 5s 60ms/step - loss: 1.3262
Epoch 3/50
80/80 [==============================] - 5s 61ms/step - loss: 1.2642
Epoch 4/50
80/80 [==============================] - 5s 61ms/step - loss: 1.2254
Epoch 5/50
80/80 [==============================] - 5s 62ms/step - loss: 1.1947
Epoch 6/50
80/80 [==============================] - 5s 61ms/step - loss: 1.1612
Epoch 7/50
80/80 [==============================] - 5s 61ms/step - loss: 1.1211
Epoch 8/50
80/80 [==============================] - 5s 61ms/step - loss: 1.0824
Epoch 9/50
80/80 [==============================] - 5s 67ms/step - loss: 1.0460
Epoch 10/50
80/80 [==============================] - 5s 63ms/step - loss: 1.0111
Epoch 11/50
80/80 [==============================] - 5s 61ms/step - loss: 0.9851
Epoch 12/50
80/80 [==============================] - 5s 60ms/step - loss: 0.9577
Epoch 13/50
80/80 [=============================

In [64]:
model.save_weights('english_to hindi.h5')

In [65]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [66]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [67]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [69]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 17ms/step
Input English sentence: mellette
Actual Hindi Translation:  मेलेट 
Predicted Hindi Translation:  मेलेट 


In [70]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 18ms/step
Input English sentence: devendra
Actual Hindi Translation:  देवेंद्र 
Predicted Hindi Translation:  देवेंद्र 


In [71]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 23ms/step
Input English sentence: celia
Actual Hindi Translation:  सेलिया 
Predicted Hindi Translation:  सेलिया 


In [72]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 19ms/step
Input English sentence: delhi junction
Actual Hindi Translation:  डेल्ही जंक्शन 
Predicted Hindi Translation:  डेल्ही जंक्शन 


In [73]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 19ms/step
Input English sentence: drupad
Actual Hindi Translation:  द्रुपद 
Predicted Hindi Translation:  द्रुपद 
